In [1]:
import json
import os
import yaml
from pathlib import Path
from dask.distributed import Client
import re

In [2]:
use_local=True

In [3]:
if not use_local:
    with open(f'~/.dlio_profiler/configuration.yaml', 'r') as file:
        dlp_yaml = yaml.safe_load(file)
        app_root = dlp_yaml["app"]
else:
    app_root = str(Path(os.getcwd()).parent.parent)

In [4]:
import sys
sys.path.insert(0, app_root)
sys.path

['/usr/WS2/haridev',
 '/usr/WS2/haridev/scr-dlio/examples',
 '/usr/workspace/iopp/projects/digio',
 '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python39.zip',
 '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python3.9',
 '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python3.9/lib-dynload',
 '',
 '/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages']

In [5]:
import dlp_analyzer
print(dlp_analyzer.__file__)
from dlp_analyzer.main import DLPAnalyzer,get_dlp_configuration,update_dlp_configuration,setup_logging,setup_dask_cluster, reset_dask_cluster, get_dlp_configuration


/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/__init__.py


In [6]:
if not use_local:
    dask_run_dir = os.path.join(app_root, "dlp_analyzer", "dask", "run_dir")
    with open (os.path.join(dask_run_dir, f"scheduler_{os.getenv('USER')}.json"), "r") as f:
        dask_scheduler = json.load(f)["address"]
else:
    dask_scheduler = None

In [7]:
app_name = "dlio_scr" # dlio dlio_scr

In [8]:
def get_conditions(json_object):
    app_io_cond = "reader" in json_object["cat"] or "checkpoint" in json_object["cat"]
    compute_cond = "compute" in json_object["name"] # Cosmoflow
    io_cond = "POSIX" == json_object["cat"] # Cosmoflow
    return app_io_cond, compute_cond, io_cond

In [9]:
conf = update_dlp_configuration(dask_scheduler=dask_scheduler, verbose=True, workers=64,
                                log_file=f"./dlp_{os.getenv('USER')}.log", rebuild_index=False, time_approximate=True, 
                                host_pattern=r'lassen(\d+)', time_granularity=30e6, skip_hostname=True, conditions=get_conditions)

In [10]:
setup_logging()

In [11]:
setup_dask_cluster()

[INFO] [12:27:57] Initialized Client with 64 workers and link http://127.0.0.1:8787/status [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:668]


In [12]:
def get_checkpoint_time(df):
    app_time = df.query("name.str.contains('PyTorchCheckpointing.checkpoint')")
    app_time_files = app_time.groupby(["pid"]).agg({"dur":sum}).max()
    return app_time_files.compute() / 1e6

def get_posix_pfs_time(df):
    posix_lustre_time = df.query("cat == 'POSIX' and filename.str.contains('/p/lustre2/haridev/dlio/scr/checkpoints')")
    posix_lustre_time_files = posix_lustre_time.groupby(["pid"]).agg({"dur":sum}).max()
    return posix_lustre_time_files.compute() / 1e6

def get_posix_ssd_time(df):
    posix_lustre_time = df.query("cat == 'POSIX' and filename.str.contains('/l/ssd/haridev/scr/checkpoints')")
    posix_lustre_time_files = posix_lustre_time.groupby(["pid"]).agg({"dur":sum}).max()
    return posix_lustre_time_files.compute() / 1e6

In [30]:
# Baseline over PFS
filename = "/g/g92/haridev/projects/scr-dlio/logs/n2_p8_base/trace*.pfw.gz"
analyzer_base_pfs = DLPAnalyzer(filename).events

[INFO] [17:12:06] Created index for 16 files [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:365]
[INFO] [17:12:06] Total size of all files are <dask.bag.core.Item object at 0x1554a8913c70> bytes [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:367]
[INFO] [17:12:07] Loading 8994 batches out of 16 files and has 147302750 lines overall [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:380]
[INFO] [17:16:28] Loaded events [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:422]
[INFO] [17:16:28] Loaded plots with slope threshold: 45 [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:428]


In [31]:
base_ckp_time = get_checkpoint_time(analyzer_base_pfs)
base_posix_lustre_time = get_posix_pfs_time(analyzer_base_pfs)
base_posix_ssd_time = get_posix_ssd_time(analyzer_base_pfs)

In [13]:
# Baseline over SSD
filename = "/usr/WS2/haridev/scr-dlio/scripts/hydra_log/megatron_deepspeed/2024-03-21-23-06-04/trace*.pfw.gz"
analyzer_base_ssd = DLPAnalyzer(filename).events

[INFO] [09:33:33] Created index for 16 files [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:365]
[INFO] [09:33:33] Total size of all files are <dask.bag.core.Item object at 0x1554aa8bd940> bytes [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:367]
[INFO] [09:33:35] Loading 8976 batches out of 16 files and has 146988160 lines overall [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:380]
[INFO] [09:38:06] Loaded events [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:422]
[INFO] [09:38:06] Loaded plots with slope threshold: 45 [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:428]


In [15]:
base_ssd_ckp_time = get_checkpoint_time(analyzer_base_ssd)
base_ssd_posix_lustre_time = get_posix_pfs_time(analyzer_base_ssd)
base_ssd_posix_ssd_time = get_posix_ssd_time(analyzer_base_ssd)


In [17]:
# SCR on SSD and with Async Flush to PFS
filename = "/g/g92/haridev/projects/scr-dlio/logs/n2_p8_scr_finer/trace*.pfw.gz"
analyzer_scr_basic = DLPAnalyzer(filename).events

[INFO] [17:33:53] Created index for 16 files [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:365]
[INFO] [17:33:53] Total size of all files are <dask.bag.core.Item object at 0x1551700ada00> bytes [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:367]
[INFO] [17:33:54] Loading 9132 batches out of 16 files and has 149432067 lines overall [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:380]
[INFO] [17:38:32] Loaded events [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:422]
[INFO] [17:38:32] Loaded plots with slope threshold: 45 [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:428]


In [18]:
scr_ssd_async_pfs_ckp_time = get_checkpoint_time(analyzer_scr_basic)
scr_ssd_async_pfs_posix_lustre_time = get_posix_pfs_time(analyzer_scr_basic)
scr_ssd_async_pfs_posix_ssd_time = get_posix_ssd_time(analyzer_scr_basic)


In [16]:
# SCR on SSD and no flush
filename = "/usr/WS2/haridev/scr-dlio/scripts/hydra_log/scr_megatron_deepspeed/2024-03-20-10-52-41/trace*.pfw.gz"
analyzer_scr_no_flush = DLPAnalyzer(filename).events

[INFO] [09:11:05] Created index for 16 files [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:365]
[INFO] [09:11:05] Total size of all files are <dask.bag.core.Item object at 0x155170c042e0> bytes [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:367]
[INFO] [09:11:05] Loading 9124 batches out of 16 files and has 149369304 lines overall [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:380]
[INFO] [09:16:34] Loaded events [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:422]
[INFO] [09:16:34] Loaded plots with slope threshold: 45 [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:428]


In [17]:

scr_ssd_no_flush_ckp_time = get_checkpoint_time(analyzer_scr_no_flush)
scr_ssd_no_flush_posix_lustre_time = get_posix_pfs_time(analyzer_scr_no_flush)
scr_ssd_no_flush_posix_ssd_time = get_posix_ssd_time(analyzer_scr_no_flush)

In [21]:
# SCR bypass
filename = "/usr/WS2/haridev/scr-dlio/scripts/hydra_log/scr_megatron_deepspeed/2024-03-21-22-44-18/trace*.pfw.gz"
analyzer_scr_bypass = DLPAnalyzer(filename).events

[INFO] [17:44:50] Created index for 16 files [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:365]
[INFO] [17:44:50] Total size of all files are <dask.bag.core.Item object at 0x15515cf35a00> bytes [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:367]
[INFO] [17:44:50] Loading 8977 batches out of 16 files and has 146963672 lines overall [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:380]
[INFO] [17:50:37] Loaded events [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:422]
[INFO] [17:50:37] Loaded plots with slope threshold: 45 [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:428]


In [22]:

scr_bypass_ckp_time = get_checkpoint_time(analyzer_scr_bypass)
scr_bypass_posix_lustre_time = get_posix_pfs_time(analyzer_scr_bypass)
scr_bypass_posix_ssd_time = get_posix_ssd_time(analyzer_scr_bypass)

In [14]:
# SCR ssd single copy
filename = "/usr/WS2/haridev/scr-dlio/scripts/hydra_log/scr_megatron_deepspeed/2024-04-08-09-33-09/trace*.pfw.gz"
analyzer_scr_ssd_single = DLPAnalyzer(filename).events

[INFO] [09:38:06] Created index for 8 files [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:365]
[INFO] [09:38:06] Total size of all files are <dask.bag.core.Item object at 0x15550f86a610> bytes [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:367]
[INFO] [09:38:07] Loading 4488 batches out of 8 files and has 73467844 lines overall [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:380]
[INFO] [09:40:17] Loaded events [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:422]
[INFO] [09:40:17] Loaded plots with slope threshold: 45 [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:428]


In [24]:

scr_ssd_single_ckp_time = get_checkpoint_time(analyzer_scr_ssd_single)
scr_ssd_single_posix_lustre_time = get_posix_pfs_time(analyzer_scr_ssd_single)
scr_ssd_single_posix_ssd_time = get_posix_ssd_time(analyzer_scr_ssd_single)

In [13]:
# SCR pfs single copy
filename = "/usr/WS2/haridev/scr-dlio/scripts/hydra_log/scr_megatron_deepspeed/2024-04-08-10-52-57/trace*.pfw.gz"
analyzer_scr_pfs_single = DLPAnalyzer(filename).events

[INFO] [12:28:24] Created index for 16 files [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:365]
[INFO] [12:28:24] Total size of all files are <dask.bag.core.Item object at 0x1554a89b8a60> bytes [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:367]
[INFO] [12:28:25] Loading 8986 batches out of 16 files and has 147099195 lines overall [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:380]
ERROR:root:Processing {"id":"6930","name":"close","cat":"POSIX","pid":"0","tid":"953748","ts":"1712599257392616","dur":"4012","ph":"X","args":{"hostname":"corona171","ret":0,"fname":"","fd":39}} failed with Invalid control character at: line 1 column 162 (char 161)
[INFO] [12:32:39] Loaded events [/usr/workspace/haridev/scr-dlio/venv/lib/python3.9/site-packages/dlp_analyzer/main.py:422]
[INFO] [12:32:39] Loaded plots with slope threshold: 45 [/usr/workspace/haridev/scr-dlio/venv/lib/pytho

In [14]:

scr_pfs_single_ckp_time = get_checkpoint_time(analyzer_scr_pfs_single)
scr_pfs_single_posix_lustre_time = get_posix_pfs_time(analyzer_scr_pfs_single)
scr_pfs_single_posix_ssd_time = get_posix_ssd_time(analyzer_scr_pfs_single)

In [15]:
scr_pfs_single_ckp_time, scr_pfs_single_posix_lustre_time, scr_pfs_single_posix_ssd_time

(dur    167.23401
 dtype: double[pyarrow],
 dur    1060.07277
 dtype: double[pyarrow],
 dur    472.086514
 dtype: double[pyarrow])

In [32]:
print(f"{str('case'):20}, {str('base'):15}, {str('base_ssd'):15}, {str('scr_ssd_a_pfs'):15}, {str('scr_ssd_nf'):15}, {str('scr_bypass'):15}, {str('scr_ssd_nf_one'):15}, {str('scr_pfs_one'):15}")
print(f"{str('checkpoint time'):20}, {float(base_ckp_time):15}, {float(base_ssd_ckp_time):15}, {float(scr_ssd_async_pfs_ckp_time):15}, {float(scr_ssd_no_flush_ckp_time):15}, {float(scr_bypass_ckp_time):15}, {float(scr_ssd_single_ckp_time):15}, {float(scr_pfs_single_ckp_time):15}")
print(f"{str('lustre time'):20}, {float(base_posix_lustre_time):15}, {0:15}, {float(scr_ssd_async_pfs_posix_lustre_time):15}, {float(scr_ssd_no_flush_posix_lustre_time):15}, {float(scr_bypass_posix_lustre_time):15}, {float(0):15}, {float(scr_pfs_single_posix_lustre_time):15}")
print(f"{str('ssd time'):20}, {0:15}, {float(base_ssd_posix_ssd_time):15}, {float(scr_ssd_async_pfs_posix_ssd_time):15}, {float(scr_ssd_no_flush_posix_ssd_time):15}, {float(scr_bypass_posix_ssd_time):15}, {float(scr_ssd_single_posix_ssd_time):15}, {float(scr_pfs_single_posix_ssd_time):15}")

case                , base           , base_ssd       , scr_ssd_a_pfs  , scr_ssd_nf     , scr_bypass     , scr_ssd_nf_one , scr_pfs_one    
checkpoint time     ,      103.057664,      126.296083,      378.215081,      349.139191,      145.122746,      153.369808,       167.23401
lustre time         ,        72.63898,               0,     1471.540838,       19.608824,       118.42119,             0.0,      1060.07277
ssd time            ,               0,       97.497736,      837.766052,      308.222378,        0.152704,      121.076746,      472.086514


### Things to try (3/27/24)
- Remove redundancy schemes: SCR_COPY_TYPE=SINGLE
  - (Potentially) Less frequent reduncy schemes:


### Things to try (4/17/24)
- Compare Base SSD with SCR SSD no flush
  - Check size
  - Check files

In [15]:
base_ssd_checkpoints = analyzer_base_ssd.query("cat == 'POSIX' and filename.str.contains('/l/ssd/haridev/scr/checkpoints')").compute()
base_ssd_checkpoints["size"].sum()/1024**3


577.07171420753

In [16]:
scr_ssd_nf_checkpoints = analyzer_scr_ssd_single.query("cat == 'POSIX' and filename.str.contains('/l/ssd/haridev/scr/checkpoints')").compute()
scr_ssd_nf_checkpoints["size"].sum()/1024**3

288.5385148981586

In [17]:
import pandas as pd
def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

### Check files and sizes

In [18]:
print_full(1000)
import re
base_ssd_checkpoints.groupby(["filename"])["size"].sum()
base_ssd_checkpoints["fname"] = base_ssd_checkpoints["filename"].apply(lambda x: str(os.path.basename(x)))
base_ssd_checkpoints["fname_regex"] = base_ssd_checkpoints["fname"].apply(lambda x: re.sub("[\d\-]+", "", x))
base_ssd_checkpoints["size_gb"] = base_ssd_checkpoints["size"] / 1024 ** 3
base_ssd_checkpoints["dur_sec"] = base_ssd_checkpoints["dur"] / 1e6
base_ssd_checkpoints.groupby(["fname_regex", "pid"])[["size_gb","dur_sec"]].sum().groupby(["fname_regex"]).max()

1000


,size_gb,dur_sec
fname_regex,,
layer.pt,37.060777,39.932368
model.pt,0.000172,0.027605
optimizer.pt,31.434213,68.512567
scr_base_megatron_deepspeed,0.0,0.000025


In [19]:
print_full(1000)
scr_ssd_nf_checkpoints.groupby(["filename"])["size"].sum()
scr_ssd_nf_checkpoints["fname"] = scr_ssd_nf_checkpoints["filename"].apply(lambda x: str(os.path.basename(x)))
scr_ssd_nf_checkpoints["fname_regex"] = scr_ssd_nf_checkpoints["fname"].apply(lambda x: re.sub("[\d\-]+", "", x))
scr_ssd_nf_checkpoints["size_gb"] = scr_ssd_nf_checkpoints["size"] / 1024 ** 3
scr_ssd_nf_checkpoints["dur_sec"] = scr_ssd_nf_checkpoints["dur"] / 1e6
scr_ssd_nf_checkpoints.groupby(["fname_regex", "pid"])[["size_gb","dur_sec"]].sum().groupby(["fname_regex"]).max()

1000


,size_gb,dur_sec
fname_regex,,
.scr,0.0,0.00023
.scrconf,0.0,0.000035
filemap_,0.002303,20.196445
flush.scr,0.000011,0.780726
halt.scr,0.0,0.720644
haridev,0.0,0.000037
index.scr,0.0,0.000028
layer.pt,37.060777,32.954393
model.pt,0.000172,0.001463


#### We see a lot of extra files which i think is expected. 
- For files which are written we see same written cost.

### Check operations for model files

In [20]:
base_ssd_checkpoints.query("fname.str.contains('model')")["name"].value_counts()

name
write         288
open64         96
__fxstat64     96
lseek64        96
close          96
Name: count, dtype: int64[pyarrow]

In [21]:
scr_ssd_nf_checkpoints.query("fname.str.contains('model')")["name"].value_counts()


name
__xstat       192
write         144
access         96
open64         48
__fxstat64     48
lseek64        48
close          48
Name: count, dtype: int64[pyarrow]

**INVESTIGATION** Why the number of write calls different smaller (1/2).

In [27]:
base_ssd_checkpoints.query("fname.str.contains('model') and name == 'write'").groupby("filename").agg({"dur":sum, "size_gb":sum, "name":"count"})

,dur,size_gb,name
filename,,,
/l/ssd/haridev/scr/checkpoints/scr_base_megatron_deepspeed/model-1-100-0.pt,43,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_base_megatron_deepspeed/model-1-100-1.pt,13419,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_base_megatron_deepspeed/model-1-100-10.pt,60,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_base_megatron_deepspeed/model-1-100-11.pt,56,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_base_megatron_deepspeed/model-1-100-12.pt,55,0.000029,3
...,...,...,...
/l/ssd/haridev/scr/checkpoints/scr_base_megatron_deepspeed/model-1-600-5.pt,160,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_base_megatron_deepspeed/model-1-600-6.pt,41,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_base_megatron_deepspeed/model-1-600-7.pt,48,0.000029,3


In [28]:
scr_ssd_nf_checkpoints.query("fname.str.contains('model') and name == 'write'").groupby("filename").agg({"dur":sum, "size_gb":sum, "name":"count"})


,dur,size_gb,name
filename,,,
/l/ssd/haridev/scr/checkpoints/scr_megatron_deepspeed/haridev/scr.defjobid/scr.dataset.1/model-1-100-0.pt,43,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_megatron_deepspeed/haridev/scr.defjobid/scr.dataset.1/model-1-100-1.pt,41,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_megatron_deepspeed/haridev/scr.defjobid/scr.dataset.1/model-1-100-2.pt,30,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_megatron_deepspeed/haridev/scr.defjobid/scr.dataset.1/model-1-100-3.pt,32,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_megatron_deepspeed/haridev/scr.defjobid/scr.dataset.1/model-1-100-4.pt,113,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_megatron_deepspeed/haridev/scr.defjobid/scr.dataset.1/model-1-100-5.pt,33,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_megatron_deepspeed/haridev/scr.defjobid/scr.dataset.1/model-1-100-6.pt,32,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_megatron_deepspeed/haridev/scr.defjobid/scr.dataset.1/model-1-100-7.pt,226,0.000029,3
/l/ssd/haridev/scr/checkpoints/scr_megatron_deepspeed/haridev/scr.defjobid/scr.dataset.2/model-1-200-0.pt,292,0.000029,3


##### Check per model file

In [29]:
base_ssd_checkpoints.query("fname.str.contains('model-1-600-0.pt')")["name"].value_counts()

name
write         3
open64        1
__fxstat64    1
lseek64       1
close         1
Name: count, dtype: int64[pyarrow]

In [53]:
scr_ssd_nf_checkpoints.query("fname.str.contains('model-1-600-0.pt')")["name"].value_counts()

name
__xstat       4
write         3
access        2
open64        1
__fxstat64    1
lseek64       1
close         1
Name: count, dtype: int64[pyarrow]

In [54]:
scr_ssd_nf_checkpoints.query("fname.str.contains('model-1-600-0.pt')")

,name,cat,pid,tid,ts,te,dur,tinterval,trange,hostname,...,io_time,app_io_time,total_time,filename,phase,size,fname,fname_regex,size_gb,dur_sec
8159,open64,POSIX,0,335916,1001811003,1001811038,35,<NA>,33.0,corona174,...,35,<NA>,35,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.000035
8160,__fxstat64,POSIX,0,335916,1001811048,1001811050,2,<NA>,33.0,corona174,...,2,<NA>,2,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.000002
8161,lseek64,POSIX,0,335916,1001811065,1001811066,1,<NA>,33.0,corona174,...,1,<NA>,1,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.000001
8162,write,POSIX,0,335916,1001811279,1001811289,10,<NA>,33.0,corona174,...,10,<NA>,10,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,320,model-1-600-0.pt,model.pt,0.0,0.00001
8163,write,POSIX,0,335916,1001811303,1001811322,19,<NA>,33.0,corona174,...,19,<NA>,19,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,30102,model-1-600-0.pt,model.pt,0.000028,0.000019
8164,write,POSIX,0,335916,1001811356,1001811360,4,<NA>,33.0,corona174,...,4,<NA>,4,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,405,model-1-600-0.pt,model.pt,0.0,0.000004
8165,close,POSIX,0,335916,1001811378,1001811388,10,<NA>,33.0,corona174,...,10,<NA>,10,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.00001
4124,access,POSIX,0,335916,1025776223,1025776228,5,<NA>,34.0,corona174,...,5,<NA>,5,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.000005
4125,__xstat,POSIX,0,335916,1025776246,1025776250,4,<NA>,34.0,corona174,...,4,<NA>,4,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.000004
4265,access,POSIX,0,335916,1025892888,1025892893,5,<NA>,34.0,corona174,...,5,<NA>,5,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.000005


In [52]:
base_ssd_checkpoints.query("fname.str.contains('model-1-600-0.pt')").groupby("name")["dur_sec"].sum()

name
__fxstat64    0.000002
close         0.000009
lseek64       0.000001
open64        0.000111
write         0.000151
Name: dur_sec, dtype: double[pyarrow]

In [55]:
scr_ssd_nf_checkpoints.query("fname.str.contains('model-1-600-0.pt')").groupby("name")["dur_sec"].sum()

name
__fxstat64    0.000002
__xstat       0.000027
access         0.00001
close          0.00001
lseek64       0.000001
open64        0.000035
write         0.000033
Name: dur_sec, dtype: double[pyarrow]

In [70]:
scr_ssd_nf_checkpoints["trange"].max()

35.0

In [71]:
base_ssd_checkpoints.query("fname.str.contains('model-1-600-0.pt')")["size"].value_counts()

size
320      1
30102    1
405      1
Name: count, dtype: int64[pyarrow]

In [72]:
scr_ssd_nf_checkpoints.query("fname.str.contains('model-1-600-0.pt')")["size"].value_counts()

size
320      1
30102    1
405      1
Name: count, dtype: int64[pyarrow]

In [73]:
scr_ssd_nf_checkpoints.query("fname.str.contains('model-1-600-0.pt')")

,name,cat,pid,tid,ts,te,dur,tinterval,trange,hostname,...,io_time,app_io_time,total_time,filename,phase,size,fname,fname_regex,size_gb,dur_sec
8159,open64,POSIX,0,335916,1001811003,1001811038,35,<NA>,33.0,corona174,...,35,<NA>,35,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.000035
8160,__fxstat64,POSIX,0,335916,1001811048,1001811050,2,<NA>,33.0,corona174,...,2,<NA>,2,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.000002
8161,lseek64,POSIX,0,335916,1001811065,1001811066,1,<NA>,33.0,corona174,...,1,<NA>,1,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.000001
8162,write,POSIX,0,335916,1001811279,1001811289,10,<NA>,33.0,corona174,...,10,<NA>,10,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,320,model-1-600-0.pt,model.pt,0.0,0.00001
8163,write,POSIX,0,335916,1001811303,1001811322,19,<NA>,33.0,corona174,...,19,<NA>,19,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,30102,model-1-600-0.pt,model.pt,0.000028,0.000019
8164,write,POSIX,0,335916,1001811356,1001811360,4,<NA>,33.0,corona174,...,4,<NA>,4,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,405,model-1-600-0.pt,model.pt,0.0,0.000004
8165,close,POSIX,0,335916,1001811378,1001811388,10,<NA>,33.0,corona174,...,10,<NA>,10,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.00001
4124,access,POSIX,0,335916,1025776223,1025776228,5,<NA>,34.0,corona174,...,5,<NA>,5,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.000005
4125,__xstat,POSIX,0,335916,1025776246,1025776250,4,<NA>,34.0,corona174,...,4,<NA>,4,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.000004
4265,access,POSIX,0,335916,1025892888,1025892893,5,<NA>,34.0,corona174,...,5,<NA>,5,/l/ssd/haridev/scr/checkpoints/scr_megatron_de...,2,<NA>,model-1-600-0.pt,model.pt,<NA>,0.000005


- Many extra operations
  - original app used 7 I/O calls across 5 ops
  - SCR has 13 I/O calls across 7 ops 

*Question* why do we do extra calls on data produced as outputs?

### Check operations for layer files

In [74]:
base_ssd_checkpoints.query("fname.str.contains('layer')")["name"].value_counts()

name
write         2640
open64         528
__fxstat64     528
lseek64        528
close          528
Name: count, dtype: int64[pyarrow]

In [75]:
scr_ssd_nf_checkpoints.query("fname.str.contains('layer')")["name"].value_counts()


name
write         1320
__xstat       1056
access         528
open64         264
__fxstat64     264
lseek64        264
close          264
Name: count, dtype: int64[pyarrow]

**INVESTIGATION** Again why do we see half the writes. 

##### Check per model file

In [76]:
base_ssd_checkpoints.query("fname.str.contains('layer-') and fname.str.contains('1-600-0.pt')")["name"].value_counts()

name
write         220
open64         44
__fxstat64     44
lseek64        44
close          44
Name: count, dtype: int64[pyarrow]

In [77]:
scr_ssd_nf_checkpoints.query("fname.str.contains('layer-') and fname.str.contains('1-600-0.pt')")["name"].value_counts()

name
write         220
__xstat       176
access         88
open64         44
__fxstat64     44
lseek64        44
close          44
Name: count, dtype: int64[pyarrow]

In [78]:
base_ssd_checkpoints.query("fname.str.contains('layer-') and fname.str.contains('1-600-0.pt')")["size"].value_counts()

size
384          44
129761280    44
64           44
20971520     44
487          44
Name: count, dtype: int64[pyarrow]

In [79]:
scr_ssd_nf_checkpoints.query("fname.str.contains('layer-') and fname.str.contains('1-600-0.pt')")["size"].value_counts()

size
384          44
129761280    44
64           44
20971520     44
487          44
Name: count, dtype: int64[pyarrow]

In [80]:
base_ssd_checkpoints.query("fname.str.contains('layer-0-1-600-0.pt')")["size"].value_counts()

size
384          1
129761280    1
64           1
20971520     1
487          1
Name: count, dtype: int64[pyarrow]

In [81]:
scr_ssd_nf_checkpoints.query("fname.str.contains('layer-0-1-600-0.pt')")["size"].value_counts()

size
384          1
129761280    1
64           1
20971520     1
487          1
Name: count, dtype: int64[pyarrow]

In [82]:
base_ssd_checkpoints.query("fname.str.contains('layer-0-1-600-0.pt')")["name"].value_counts()

name
write         5
open64        1
__fxstat64    1
lseek64       1
close         1
Name: count, dtype: int64[pyarrow]

In [83]:
scr_ssd_nf_checkpoints.query("fname.str.contains('layer-0-1-600-0.pt')")["name"].value_counts()

name
write         5
__xstat       4
access        2
open64        1
__fxstat64    1
lseek64       1
close         1
Name: count, dtype: int64[pyarrow]

Same Behavior here.

# Next Steps
1. Check why PFS is so much cost in SCR. We looked at flush being the bulk of the cost.
2. INVESTIGATE: Without async I/O the ssd is 121.076746 and with async I/O is 472.086514

In [16]:
scr_pfs_single_ckp_time, scr_pfs_single_posix_lustre_time, scr_pfs_single_posix_ssd_time

(dur    167.23401
 dtype: double[pyarrow],
 dur    1060.07277
 dtype: double[pyarrow],
 dur    472.086514
 dtype: double[pyarrow])

In [18]:
def get_checkpoint_time(df):
    app_time = df.query("name.str.contains('PyTorchCheckpointing.checkpoint')")
    app_time_files = app_time.groupby(["pid"]).agg({"dur":sum}).max()
    return app_time_files.compute() / 1e6

def get_posix_pfs(df):
    posix_lustre_time = df.query("cat == 'POSIX' and filename.str.contains('/p/lustre2/haridev/dlio/scr/checkpoints')")
    return posix_lustre_time

def get_posix_ssd_time(df):
    posix_lustre_time = df.query("cat == 'POSIX' and filename.str.contains('/l/ssd/haridev/scr/checkpoints')")
    posix_lustre_time_files = posix_lustre_time.groupby(["pid"]).agg({"dur":sum}).max()
    return posix_lustre_time_files.compute() / 1e6

In [28]:
scr_pfs_single_posix_lustre_df = get_posix_pfs(analyzer_scr_pfs_single)
scr_pfs_single_posix_lustre_df.groupby(["name","pid"]).agg({"dur":sum}).groupby("name").max().compute() / 1e6, scr_pfs_single_posix_lustre_time

(                   dur
 name                  
 __lxstat      1.416893
 __xstat       0.314929
 __xstat64     0.001076
 access        0.023796
 chmod         1.043387
 chown         1.463341
 close            1.201
 fsync        10.568369
 ftruncate     0.015173
 lseek         0.000066
 mkdir         0.010559
 open         12.207076
 read          0.114711
 write      1032.015623,
 dur    1060.07277
 dtype: double[pyarrow])

In [32]:
scr_pfs_single_posix_lustre_df.groupby(["name"]).agg({"size":sum}).compute() /1024 **3

,size
name,
mkdir,0.0
__xstat64,0.0
access,0.0
open,0.0
write,576.791053
fsync,0.0
close,0.0
read,0.000106
lseek,0.0


In [33]:
analyzer_base_pfs_posix_lustre_df = get_posix_pfs(analyzer_base_pfs)
analyzer_base_pfs_posix_lustre_df.groupby(["name"]).agg({"size":sum}).compute() /1024 **3

,size
name,
mkdir,0.0
__xstat64,0.0
open64,0.0
__fxstat64,0.0
lseek64,0.0
write,577.071714
close,0.0


In [34]:
analyzer_base_pfs_posix_lustre_df.groupby(["name","pid"]).agg({"dur":sum}).groupby("name").max().compute() / 1e6

,dur
name,
__fxstat64,0.156578
__xstat64,0.005139
close,0.179031
lseek64,0.000629
mkdir,0.005487
open64,2.708844
write,69.628272


In [41]:
analyzer_base_pfs_posix_lustre_df.compute().groupby(["name"]).agg({"size": "median"}) / 1024 **2

,size
name,
__fxstat64,<NA>
__xstat64,<NA>
close,<NA>
lseek64,<NA>
mkdir,<NA>
open64,<NA>
write,0.000671


In [45]:
scr_pfs_single_posix_lustre_df.compute().query("name == 'write'")["size"].describe()

count            18936.0
mean     32706203.922634
std       4315202.438485
min                 28.0
25%           33554432.0
50%           33554432.0
75%           33554432.0
max           33554432.0
Name: size, dtype: double[pyarrow]

In [46]:
analyzer_base_pfs_posix_lustre_df.compute().query("name == 'write'")["size"].describe()

count              3792.0
mean     163403490.240506
std      392716773.891337
min                  64.0
25%                 384.0
50%                 704.0
75%           129761280.0
max          1875123200.0
Name: size, dtype: double[pyarrow]